# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 20 élément pris au hasard
import json
import random
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=20)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_value_2,spatial_value_3,spatial_value_4,temporal,temporal_key_0,temporal_key_1,temporal_key_2,temporal_value_0,temporal_value_1,temporal_value_2
0,00039882,plaza,entrance_hall,train_station/platform,archive,subway_station/platform,0.322113,0.140566,0.096089,0.052570,...,NaN,NaN,NaN,NaN,TIMEX_MISC,TIMEX_DATE,TIMEX_MISC,La Foire commerciale internationale de Bruxelles,6 . IV.1932,la Foire
1,00009735,library/outdoor,hotel/outdoor,apartment_building/outdoor,building_facade,hospital,0.326709,0.182635,0.142580,0.106532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00037962,palace,campus,castle,embassy,building_facade,0.757402,0.059692,0.049110,0.036610,...,NaN,NaN,NaN,NaN,TIMEX_MISC,TIMEX_DATE,NaN,Revue du printemps,8/4/1940,NaN
3,00029448,street,ski_resort,residential_neighborhood,alley,ice_floe,0.339991,0.235848,0.067254,0.034559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00009560,church/outdoor,castle,tower,palace,building_facade,0.292801,0.264314,0.242183,0.073323,...,NaN,NaN,NaN,NaN,TIMEX_DATE,NaN,NaN,mai 1940,NaN,NaN


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
data.head()

beeldid         spatial                            
                      value         key           value
0  00009735           Liège   LOC_PLACE           Liège
1  00037962       Bruxelles   LOC_PLACE       Bruxelles
2  00029448            Mons   LOC_PLACE            Mons
3  00005542       Bruxelles   LOC_PLACE       Bruxelles
4  00005542  la grand-place  LOC_STREET  la grand-place

In [9]:
data.columns = ['_'.join(col) for col in data.columns]

In [10]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [11]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value
0,00009735,Liège,LOC_PLACE,Liège
1,00037962,Bruxelles,LOC_PLACE,Bruxelles
2,00029448,Mons,LOC_PLACE,Mons
3,00005542,Bruxelles,LOC_PLACE,Bruxelles
4,00005542,la grand-place,LOC_STREET,la grand-place


In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name
0,00009735,Liège,LOC_PLACE,Liège,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398.0,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège
1,00037962,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139.0,NaN,NaN,NaN,NaN,NaN
2,00029448,Mons,LOC_PLACE,Mons,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754.0,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon
3,00005542,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206.0,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels
4,00005542,la grand-place,LOC_STREET,la grand-place,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206.0,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels


In [15]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [16]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00009735,Liège,LOC_PLACE,Liège,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398.0,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,LOC_PLACE,Q56061,administrative territorial entity
1,00037962,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139.0,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
2,00029448,Mons,LOC_PLACE,Mons,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754.0,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,LOC_PLACE,Q56061,administrative territorial entity
3,00005542,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206.0,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
4,00005542,la grand-place,LOC_STREET,la grand-place,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206.0,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,LOC_STREET,Q83620,thoroughfare


In [17]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [18]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [19]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00009735,Liège,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,Q56061,administrative territorial entity
1,00037962,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00029448,Mons,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,Q56061,administrative territorial entity
3,00005542,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
4,00005542,la grand-place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare


In [20]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [21]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [22]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00009735,Liège,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,Q56061,administrative territorial entity
1,00037962,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00029448,Mons,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,Q56061,administrative territorial entity
3,00005542,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
4,00005542,grand-place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare


In [23]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [24]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [25]:
data

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00009735,liege,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,Q56061,administrative territorial entity
1,00037962,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00029448,mons,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,Q56061,administrative territorial entity
3,00005542,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
4,00005542,grandplace,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare
5,00040319,place royale,LOC_SPACES,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q294440,public space
6,00040319,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
7,00040319,congo,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
8,00040319,belgique,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
9,00040319,congo,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity


In [63]:
gb = data.groupby(['beeldid'])
gb.head() 

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00009735,liege,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,Q56061,administrative territorial entity
1,00037962,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Revue du printemps à Bruxelles. Les troupes dé...,"Le roi Léopold III, [...-1939]. - 573 photos",3139,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00029448,mons,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,Q56061,administrative territorial entity
3,00005542,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
4,00005542,grandplace,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare
5,00040319,place royale,LOC_SPACES,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q294440,public space
6,00040319,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
7,00040319,congo,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
8,00040319,belgique,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity
9,00040319,congo,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Expédition automobile Belgique-Congo. La deux...,"Diverses photos de la Belgique avant-guerre, [...",43993,medailles et decorations--belgique,"Q974,Q31","Belgium,Democratic Republic of the Congo",NaN,NaN,Q56061,administrative territorial entity


In [55]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [56]:
gb_filtered.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00009735,liege,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Liège. Vieille maison Rue de la Goffe. [Frei g...,"Vues de Liège, 1940-1945. - 34 photos",398,liege (belgique)--photographies,Q31,Belgium,Q3992,Liège,Q56061,administrative territorial entity
2,00029448,mons,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Arrivée des premiers tanks de la 3e D. blindée...,"Libération de Hyon par les alliés, 9/1944. - 4...",1754,hyon (belgique)--histoire--1944 (liberation),Q31,Belgium,Q2360484,Hyon,Q56061,administrative territorial entity
3,00005542,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
4,00005542,grandplace,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"""Vodden en Benen"". Ce cri familier des rues de...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare
11,00005409,avenue louise,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,La jolie Bruxelloise. Deux jeunes femmes se pr...,"Reportage La jolie Bruxelloise, 1940-1945. - 1...",206,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels,Q83620,thoroughfare
12,00009475,loge maconnique de bruges,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"La loge maconnique de Bruges, qui après celles...","Vues de Bruges, 1940-1945. - 65 photos",133408,NaN,Q31,Belgium,Q12994,Bruges,Q41176,building
13,00009475,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"La loge maconnique de Bruges, qui après celles...","Vues de Bruges, 1940-1945. - 65 photos",133408,NaN,Q31,Belgium,Q12994,Bruges,Q56061,administrative territorial entity
14,00009475,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"La loge maconnique de Bruges, qui après celles...","Vues de Bruges, 1940-1945. - 65 photos",133408,NaN,Q31,Belgium,Q12994,Bruges,Q56061,administrative territorial entity
15,00000715,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Bruxelles. Après la reddition, les troupes all...","Troupes allemandes à Bruxelles, 5/1940. - 10 p...",5,campagne des 18 jours (1940)--bruxelles (belgi...,Q31,Belgium,Q239,Brussels,Q56061,administrative territorial entity
16,00000715,grandplace,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"Bruxelles. Après la reddition, les troupes all...","Troupes allemandes à Bruxelles, 5/1940. - 10 p...",5,campagne des 18 jours (1940)--bruxelles (belgi...,Q31,Belgium,Q239,Brussels,Q83620,thoroughfare


In [65]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
gb.head()


,beeldid,spatial_value
0,00000715,bruxelles grandplace
1,00005409,avenue louise
2,00005542,bruxelles grandplace
3,00009475,loge maconnique de bruges anvers bruxelles
4,00009735,liege


In [71]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "country_name", "loc_name"]])


In [72]:
#on splitte les noms de pays multiples
gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [74]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [75]:
gb

,beeldid,spatial_value,country_name,loc_name
0,00000715,bruxelles grandplace,Belgium,Brussels
8,00005409,avenue louise,Belgium,Brussels
9,00005542,bruxelles grandplace,Belgium,Brussels
17,00009475,loge maconnique de bruges anvers bruxelles,Belgium,Bruges
44,00009735,liege,Belgium,Liège
45,00009933,ypres grandplace,Belgium,Ypres
53,00011504,anvers,Belgium,Antwerp
54,00017628,gare du nord allemagne,Belgium,Brussels
62,00028265,camp de beverloo,Belgium,"Saint-Gilles - Sint-Gillis,Verviers"
63,00029091,bruxelles place du trone,Belgium,Brussels


In [79]:
#appliquer get_nominatim. ATTENTION : ajouter le pays
import requests
import requests_cache
from time import sleep

# Nominatim bloque les requêtes répétées
requests_cache.install_cache('nominatim_cache')

s = requests.Session()

def get_nominatim(value, countrycodes=[''], limit=5, lang="fr"):
    # doc : https://wiki.openstreetmap.org/wiki/Nominatim
    url = 'http://nominatim.openstreetmap.org/'
    

    params = {'q': value,
              'format': 'jsonv2',
              'addressdetails': 1,
              'limit': limit,
              'email': 'ettorerizza@mail.com',
              'polygon_kml': 0,
              'extratags': 1,
              'namedetails': 0,
              # ajouter un country_code vide simule 
              # la péréférence pour un pays de geopy
              # sans se limiter à celui-ci
              'countrycodes': countrycodes,
              'accept-language': lang
              }

    headers = {
        'User-Agent': 'Ugesco app',
        'From': 'ettorerizza@mail.com'
    }

    result = s.get(url, params=params, headers=headers, timeout=10).json()

    # Nominatim bloque les requêtes trop rapides
    sleep(1)

    return result

gb['nominatim'] = np.vectorize(get_nominatim)(gb['spatial_value'], [x for x in gb['country_name']])

In [80]:
gb.head(10)

,beeldid,spatial_value,country_name,loc_name,nominatim,test
0,00000715,bruxelles grandplace,Belgium,Brussels,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi..."
8,00005409,avenue louise,Belgium,Brussels,"[{'place_id': '74641228', 'licence': 'Data © O...","Avenue Louise, Les Gardes, Royan, Rochefort, C..."
9,00005542,bruxelles grandplace,Belgium,Brussels,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi..."
17,00009475,loge maconnique de bruges anvers bruxelles,Belgium,Bruges,[],None
44,00009735,liege,Belgium,Liège,"[{'place_id': '179318246', 'licence': 'Data © ...","Liège, Wallonie, 4000, Belgique"
45,00009933,ypres grandplace,Belgium,Ypres,"[{'place_id': '142026336', 'licence': 'Data © ...","President, Case Dondolini, Santa Fiora, Unione..."
53,00011504,anvers,Belgium,Antwerp,"[{'place_id': '179011988', 'licence': 'Data © ...","Anvers, Flandre, Belgique"
54,00017628,gare du nord allemagne,Belgium,Brussels,"[{'place_id': '8365152', 'licence': 'Data © Op...","Gare du Nord, 123, Jülicher Straße, Rehmvierte..."
62,00028265,camp de beverloo,Belgium,"Saint-Gilles - Sint-Gillis,Verviers",[],None
63,00029091,bruxelles place du trone,Belgium,Brussels,"[{'place_id': '162155031', 'licence': 'Data © ...","Place du Trône, Quartier Royal, Bruxelles, Vil..."


In [81]:
#parse the first nominatim json
import json
def get_element(array):
    for el in array:
        return el['display_name'] 
    
gb['test'] = gb['nominatim'].apply(lambda x: get_element(x))
gb.head(20)

,beeldid,spatial_value,country_name,loc_name,nominatim,test
0,00000715,bruxelles grandplace,Belgium,Brussels,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi..."
8,00005409,avenue louise,Belgium,Brussels,"[{'place_id': '74641228', 'licence': 'Data © O...","Avenue Louise, Les Gardes, Royan, Rochefort, C..."
9,00005542,bruxelles grandplace,Belgium,Brussels,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi..."
17,00009475,loge maconnique de bruges anvers bruxelles,Belgium,Bruges,[],None
44,00009735,liege,Belgium,Liège,"[{'place_id': '179318246', 'licence': 'Data © ...","Liège, Wallonie, 4000, Belgique"
45,00009933,ypres grandplace,Belgium,Ypres,"[{'place_id': '142026336', 'licence': 'Data © ...","President, Case Dondolini, Santa Fiora, Unione..."
53,00011504,anvers,Belgium,Antwerp,"[{'place_id': '179011988', 'licence': 'Data © ...","Anvers, Flandre, Belgique"
54,00017628,gare du nord allemagne,Belgium,Brussels,"[{'place_id': '8365152', 'licence': 'Data © Op...","Gare du Nord, 123, Jülicher Straße, Rehmvierte..."
62,00028265,camp de beverloo,Belgium,"Saint-Gilles - Sint-Gillis,Verviers",[],None
63,00029091,bruxelles place du trone,Belgium,Brussels,"[{'place_id': '162155031', 'licence': 'Data © ...","Place du Trône, Quartier Royal, Bruxelles, Vil..."
